****
# Import modules
****

In [1]:
import os
import pandas as pd
import numpy as np

import statsmodels.formula.api as smf

****
# Load Data
****

In [2]:
file_path = "C:\\Users\\kruu\\store\\data_LSGG\\"

filenames = next(os.walk(file_path), (None, None, []))[2]

data_LSGG = pd.read_parquet(os.path.join(file_path +filenames[0]))

for path in filenames[1:]:
    temp = pd.read_parquet(os.path.join(file_path + path))
    data_LSGG = pd.concat((data_LSGG, temp))
    

In [4]:
#Check any missing data
data_LSGG[["avg_wind_dir", "avg_wind_speed", "avg_vis", "avg_temp", "avg_press"]].isna().any()

#droping missing values if needed
data_LSGG = data_LSGG.dropna()

****
# LR Model
****

In [ ]:
model_lm = smf.ols(
    "nominal_distance_prop ~ C(star) + C(weekday) + avg_vis + avg_wind_speed + C(season) + avg_vis + avg_press + C(body_type, Treatment(reference='Narrowbody')) + C(rush_hour) + avg_temp", 
    data_LSGG, 
)

res_lm = model_lm.fit()
res_lm.summary()

****
# Plots
****

In [ ]:
boxplot = data_LSGG.boxplot(["nominal_distance_prop"], by = ["star"],
                     figsize = (16, 9),
                     vert = False,
                     showmeans = False,
                     notch = False,
                     whis = (2.5,97.5))

boxplot.axvline(x=1, color='darkorange', linestyle=':', linewidth=2)

custom_labels = ['AKIT3R', 'BELU3N', 'KINE2N', 'LUSA2N']
boxplot.set_yticklabels(custom_labels, fontsize=22, rotation=45)

boxplot.get_figure().suptitle('')
boxplot.set_ylabel("")
boxplot.set_title('')

boxplot.set_axisbelow(True)
boxplot.grid(True, linestyle='--', alpha=0.7)
boxplot.spines['top'].set_visible(False)
boxplot.spines['right'].set_visible(False)
boxplot.spines['left'].set_visible(False)
boxplot.spines['bottom'].set_visible(False)
boxplot.tick_params(axis='x', which='both', length=0, labelsize=20)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

coefficients = res_lm.params
p_values = res_lm.pvalues

# features = coefficients.index  # Feature names
features = np.array([ #Same order as coefficients.index 
    "Intercept", 
    "STAR: BELU3N",
    "STAR: KINE2N",
    "STAR: LUSA2N",
    "WEEKDAY: Tuesday",
    "WEEKDAY: Wednesday",
    "WEEKDAY: Thursday",
    "WEEKDAY: Friday",
    "WEEKDAY: Saturday",
    "WEEKDAY: Sunday",
    "SEASON: Spring",
    "SEASON: Summer",
    "SEASON: Winter",
    "BODY TYPE: Regional jet",
    "RUSH HOUR: True",
    "Visibility",
    "Wind speed",
    "Pressure",
    "Temperature",
])
importance = coefficients.values  # Absolute coefficients to represent feature importance

indices = np.argsort(importance)

bar_color = 'steelblue'

def significance_stars(p_value):
    if p_value <= 0.01:
        return '***'
    elif p_value <= 0.05:
        return '**'
    elif p_value <= 0.10:
        return '*'
    else:
        return ''

plt.figure(figsize=(12, 8))
plt.barh(range(len(importance)), importance[indices], align='center', color=bar_color)

# Add significance stars and values
for i in range(len(importance)):
    pvalue = significance_stars(p_values[indices][i])
    if p_values[indices][i] < 0.05: 
        if importance[indices][i] > 0:
            plt.text(importance[indices][i] * 1.01, i, f'{np.round(importance[indices][i],3)}{pvalue}', va='center', fontsize=14)
        else:
            plt.text(0.001, i, f'{np.round(importance[indices][i],3)}{pvalue}', va='center', fontsize=14)
        
# Set y-ticks and color them based on significance
y_labels = plt.yticks(range(len(importance)), features[indices], fontsize=16)
for i in range(len(importance)):
    if p_values[indices][i] > 0.05:
        y_labels[1][i].set_color('firebrick')

# Center the zero point in the x-axis
ax = plt.gca()
ax.set_xlim(-0.3, 0.8)
ax.grid(True, axis='y', linestyle='--', alpha=0.7)
ax.set_axisbelow(True)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.tick_params(axis='y', which='both', length=0)
plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False) 
plt.xlabel('')
plt.title('')
plt.tight_layout()
plt.show()


****
# Linear mixed model
****

In [ ]:
# The effect of these predictors is assumed to be constant across all observations.
fixed_effect = "nominal_distance_prop ~ C(body_type, Treatment(reference='Narrowbody')) + C(weekday) + avg_vis + avg_wind_speed + avg_press + rush_hour" 

# Randome effect at STAR level
random_effect = "~ avg_vis + avg_wind_speed + C(weekday)"

model_mixed = smf.mixedlm(
    fixed_effect, 
    data_LSGG,
    groups = "star", 
    re_formula=random_effect
)

res_mixedlm = model_mixed.fit()
res_mixedlm.summary()